In [ ]:
repo_url = 'https://github.com/ZZIDZZ/tensorflow-object-detection.git'
# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 10000  # 200000 to improve

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [3]:
!dir

 Volume in drive D has no label.
 Volume Serial Number is 09EC-147D

 Directory of d:\Projects\pyproj\tensorflow-object-detection

02/23/2022  06:44 AM    <DIR>          .
02/23/2022  06:44 AM    <DIR>          ..
02/23/2022  06:04 AM                70 .gitignore
02/23/2022  06:44 AM               332 asnu
02/23/2022  06:38 AM    <DIR>          data
02/23/2022  06:04 AM    <DIR>          deploy
02/23/2022  06:04 AM             4,948 generate_tfrecord.py
02/23/2022  06:04 AM             1,659 LICENSE
02/23/2022  06:04 AM         1,827,289 local_inference_test.ipynb
02/23/2022  06:04 AM             6,403 local_inference_test.py
02/23/2022  06:04 AM             3,778 README.md
02/23/2022  06:04 AM                29 requirements.txt
02/23/2022  06:04 AM             1,810 resize_images.py
02/23/2022  06:04 AM         2,021,604 tensorflow_object_detection_training_colab.ipynb
02/23/2022  06:04 AM    <DIR>          test
02/23/2022  06:44 AM               339 TRAININGTRAINING.ipynb
02/23/2022 

In [ ]:
import os
os.environ['PYTHONPATH'] += ':D:\\Projects\\pyproj\models\\research:D:\\Projects\\pyproj\models\\researchslim\\'